In [1]:
# 결과 확인을 용이하게 하기 위한 코드
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

- [04 PyTorch에서 일반적인 checkpoint 저장하기 & 불러오기](https://tutorials.pytorch.kr/recipes/recipes/saving_and_loading_a_general_checkpoint.html)
- 추론(inference) 또는 학습(training)의 재개를 위해 체크포인트(checkpoint) 모델을 저장하고 불러오는 것은 마지막으로 중단했던 부분을 선택하는데 도움을 줄 수 있음
- 체크포인트를 저장할 때는 단순히 모델의 state_dict 이상의 것을 저장해야 함
- 모델 학습 중에 갱신되는 퍼버와 매개변수들을 포함하는 옵티마이저(Optimizer)의 state_dict를 함께 저장하는 것이 중요함
- 이 외에도 중단 시점의 에포크(epoch), 마지막으로 기록된 학습 오차(training loss), 외부 torch.nn.Embedding 계층 등, 알고리즘에 따라 저장하고 싶은 항목들이 있을 것임

# 개요
- 여러 체크포인트들을 저장하기 위해서는 사전(dictionary)에 체크포인트들을 구성하고 torch.save()를 사용하여 사전을 직렬화(serialize)해야 함
- 일반적인 PyTorch에서는 이러한 여러 체크포인트들을 저장할 때 .tar 확장자를 사용하는 것이 일반적인 규칙임
- 항목들을 불러올 때에는, 먼저 모델과 옵티마이저를 초기화하고, torch.load()를 사용하여 사전을 불러옴
- 이후 원하는대로 저장한 항목들을 사전에 조회하여 접근할 수 있습니다.
- 이 레시피에서는 여러 체크포인트들을 어떻게 저장하고 불러오는지 살펴보겠습니다.

# 단계
1. 데이터 불러올 때 필요한 라이브러리들 불러오기
2. 신경망을 구성하고 초기화하기
3. 옵티마이저 초기화하기
4. 일반적인 체크포인트 저장하기
5. 일반적인 체크포인트 불러오기

## 데이터 불러올 때 필요한 라이브러리들 불러오기
- `torch`와 여기 포함된 `torch.nn`과 `torch.optim`을 사용할 것임

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

## 신경망을 구성하고 초기화하기

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()
print(net)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


## 옵티마이저 초기화하기
- momentum을 갖는 SGD를 사용하기

In [4]:
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

## 일반적인 체크포인트 저장하기
- 관련된 모든 정보들을 모아서 사전을 구성하기

In [5]:
# 추가 정보
EPOCH = 5
PATH = 'model.pt'
LOSS = 0.4

torch.save({'epoch': EPOCH,
            'model_state_dict': net.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': LOSS}, PATH)

## 일반적인 체크포인트 불러오기
- 먼저 모델과 옵티마이저를 초기화한 뒤, 사전을 불러오는 것을 기억하기!

In [16]:
model = Net()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

model.eval()
# 또는
model.train()

<All keys matched successfully>

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

- 추론(inference)을 실행하기 전에 `model.eval()`을 호출하여 dropout과 batch normalization layer를 evaluation 모드로 바꾸기
    - 이것을 빼먹으면 일관성 없는 추론 결과를 얻게 됨
- 만약 학습을 계속하길 원한다면 `model.train()`을 호출하여 이 층(layer)들이 학습 모드인지 확인(ensure)하기